# Análisis de Datos Futbolísticos en SQL

### Importaciones

In [1]:
import numpy as np 
import pandas as pd 
import sqlite3

### Conexión con la Base de Datos

In [2]:
conn = sqlite3.connect('database.sqlite')

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


In [3]:
def consulta(texto):
    return pd.read_sql(texto, conn)

### Consultas

##### Países y Ligas en la Base de Datos

In [4]:
consulta('SELECT c.name AS País, l.name AS Liga FROM Country AS c JOIN League AS l ON c.id = l.country_id')

,País,Liga
0,Belgium,Belgium Jupiler League
1,England,England Premier League
2,France,France Ligue 1
3,Germany,Germany 1. Bundesliga
4,Italy,Italy Serie A
5,Netherlands,Netherlands Eredivisie
6,Poland,Poland Ekstraklasa
7,Portugal,Portugal Liga ZON Sagres
8,Scotland,Scotland Premier League
9,Spain,Spain LIGA BBVA


##### Número de partidos, equipos y jugadores

In [5]:
print(consulta('SELECT COUNT(*) AS n_partidos FROM Match;'))
print(consulta('SELECT COUNT(*) AS n_equipos FROM Team;'))
print(consulta('SELECT COUNT(*) AS n_jugadores FROM Player;'))

   n_partidos
0       25979
   n_equipos
0        299
   n_jugadores
0        11060


##### Mejores Jugadores

In [6]:
consulta('''SELECT 
                Player.player_name AS Jugador,
                t.rating AS Valoración
            FROM (
                SELECT player_api_id, MAX(overall_rating) AS rating FROM Player_Attributes
                GROUP BY player_api_id
            ) AS t
            JOIN Player ON t.player_api_id = Player.player_api_id
            ORDER BY rating DESC
            LIMIT 10;''')

,Jugador,Valoración
0,Lionel Messi,94
1,Gianluigi Buffon,93
2,Wayne Rooney,93
3,Cristiano Ronaldo,93
4,Xavi Hernandez,92
5,Gregory Coupet,92
6,Thierry Henry,91
7,John Terry,91
8,Iker Casillas,91
9,Alessandro Nesta,91


##### Jugadores con mayor potencial

In [7]:
consulta('''SELECT 
                Player.player_name AS Jugador,
                t.potential AS Potencial
            FROM (
                SELECT player_api_id, MAX(potential) AS potential FROM Player_Attributes
                GROUP BY player_api_id
            ) AS t
            JOIN Player ON t.player_api_id = Player.player_api_id
            ORDER BY potential DESC
            LIMIT 10;''')

,Jugador,Potencial
0,Lionel Messi,97
1,Ronaldinho,95
2,Cristiano Ronaldo,95
3,Neymar,94
4,Robinho,94
5,Zlatan Ibrahimovic,94
6,Thierry Henry,93
7,Joe Cole,93
8,Gianluigi Buffon,93
9,Andrea Pirlo,93


##### Jugadores con 75 o más de media con mayor margen de mejora

In [8]:
consulta('''SELECT 
                Player.player_name AS Jugador,
                t.mejora AS Mejora
            FROM (
                SELECT player_api_id, MAX(potential-overall_rating) AS mejora FROM Player_Attributes
                WHERE overall_rating >= 75
                GROUP BY player_api_id
            ) AS t
            JOIN Player ON t.player_api_id = Player.player_api_id
            ORDER BY mejora DESC
            LIMIT 10;''')

,Jugador,Mejora
0,Theo Walcott,17
1,Yoann Gourcuff,17
2,Giovani dos Santos,17
3,Sergio Aguero,16
4,Bojan Krkic,16
5,Lulinha,16
6,Piotr Trochowski,15
7,Alou Diarra,15
8,Thiago Silva,15
9,Breno,15


##### Partido con mayor diferencia de goles

In [9]:
consulta('''SELECT
                t_home.team_long_name AS nombre_equipo_local, 
                m.home_team_goal AS goles_equipo_local, 
                m.away_team_goal AS goles_equipo_visitante,  
                t_away.team_long_name AS nombre_equipo_visitante,
                l.name AS liga
            FROM Match AS m
            JOIN Team AS t_home ON m.home_team_api_id = t_home.team_api_id
            JOIN Team AS t_away ON m.away_team_api_id = t_away.team_api_id
            JOIN League AS l ON m.league_id = l.id
            ORDER BY ABS(m.home_team_goal - m.away_team_goal) DESC
            LIMIT 1;''')

,nombre_equipo_local,goles_equipo_local,goles_equipo_visitante,nombre_equipo_visitante,liga
0,PSV,10,0,Feyenoord,Netherlands Eredivisie


##### Partido con más goles

In [10]:
consulta('''SELECT
                t_home.team_long_name AS nombre_equipo_local, 
                m.home_team_goal AS goles_equipo_local, 
                m.away_team_goal AS goles_equipo_visitante,  
                t_away.team_long_name AS nombre_equipo_visitante,
                l.name AS liga
            FROM Match AS m
            JOIN Team AS t_home ON m.home_team_api_id = t_home.team_api_id
            JOIN Team AS t_away ON m.away_team_api_id = t_away.team_api_id
            JOIN League AS l ON m.league_id = l.id
            ORDER BY (home_team_goal + away_team_goal) DESC
            LIMIT 1;''')

,nombre_equipo_local,goles_equipo_local,goles_equipo_visitante,nombre_equipo_visitante,liga
0,Motherwell,6,6,Hibernian,Scotland Premier League


##### Equipos más goleadores

In [11]:
consulta('''SELECT 
                t.team_long_name AS Equipo,
                ROUND(AVG(m.goles), 2) AS Goles_por_partido
            FROM (
                SELECT 
                home_team_api_id AS id,
                AVG(home_team_goal) AS goles
                FROM Match
                GROUP BY home_team_api_id

                UNION ALL

                SELECT 
                away_team_api_id AS id,
                AVG(away_team_goal) AS goles
                FROM Match
                GROUP BY away_team_api_id
            ) AS m
            JOIN Team AS t ON m.id = t.team_api_id
            GROUP BY m.id
            ORDER BY Goles_por_partido DESC
            LIMIT 10;''')

,Equipo,Goles_por_partido
0,FC Barcelona,2.79
1,Real Madrid CF,2.77
2,FC Bayern Munich,2.40
3,PSV,2.40
4,Ajax,2.38
5,Celtic,2.29
6,SL Benfica,2.29
7,FC Porto,2.18
8,FC Basel,2.16
9,Rangers,2.13


##### Equipos que más goles encajan

In [12]:
consulta('''SELECT 
                t.team_long_name AS Equipo,
                ROUND(AVG(m.goles), 2) AS Goles_encajados
            FROM (
                SELECT 
                away_team_api_id AS id,
                SUM(home_team_goal) AS goles
                FROM Match
                GROUP BY home_team_api_id

                UNION ALL

                SELECT 
                home_team_api_id AS id,
                SUM(away_team_goal) AS goles
                FROM Match
                GROUP BY away_team_api_id
            ) AS m
            JOIN Team AS t ON m.id = t.team_api_id
            GROUP BY m.id
            ORDER BY Goles_encajados DESC
            LIMIT 10''')

,Equipo,Goles_encajados
0,Real Valladolid,350.00
1,Portsmouth,313.50
2,Bologna,271.00
3,Sparta Rotterdam,261.00
4,SM Caen,249.00
5,Newcastle United,247.00
6,Hamburger SV,244.33
7,UD Almería,234.00
8,Neuchâtel Xamax,234.00
9,Heracles Almelo,231.75


##### Equipos con mayor número de porterías a cero

In [13]:
consulta('''SELECT 
                t.team_long_name AS Equipo,
                COUNT(*) AS Porterias_a_cero
            FROM (
                SELECT 
                away_team_api_id AS id,
                match_api_id
                FROM Match
                WHERE home_team_goal = 0

                UNION ALL

                SELECT 
                home_team_api_id AS id,
                match_api_id
                FROM Match
                WHERE away_team_goal = 0
            ) AS m
            JOIN Team AS t ON m.id = t.team_api_id
            GROUP BY Equipo
            ORDER BY Porterias_a_cero DESC
            LIMIT 10;''')

,Equipo,Porterias_a_cero
0,Celtic,153
1,FC Barcelona,140
2,Manchester United,133
3,Juventus,132
4,FC Porto,128
5,Atlético Madrid,125
6,FC Bayern Munich,124
7,Chelsea,123
8,Ajax,122
9,Paris Saint-Germain,120


##### Promedio de Goles por Liga y Temporada

In [14]:
consulta('''SELECT 
                l.name AS Liga, 
                m.season AS Temporada,
                COUNT(DISTINCT m.stage) AS N_jornadas,
                ROUND(AVG(m.home_team_goal + m.away_team_goal), 2) AS goles_por_partido
            FROM Match AS m
            JOIN League AS l ON m.league_id = l.id
            JOIN Country AS c ON l.country_id = c.id
            WHERE c.name IN ('Spain', 'England', 'Germany', 'Italy', 'France')
            GROUP BY league_id, season''')

,Liga,Temporada,N_jornadas,goles_por_partido
0,England Premier League,2008/2009,38,2.48
1,England Premier League,2009/2010,38,2.77
2,England Premier League,2010/2011,38,2.80
3,England Premier League,2011/2012,38,2.81
4,England Premier League,2012/2013,38,2.80
5,England Premier League,2013/2014,38,2.77
6,England Premier League,2014/2015,38,2.57
7,England Premier League,2015/2016,38,2.70
8,France Ligue 1,2008/2009,38,2.26
9,France Ligue 1,2009/2010,38,2.41


##### Resultado de todos los Clásicos (Real Madrid CF - FC Barcelona)

In [15]:
consulta('''SELECT 
                t_home.team_long_name AS Local, 
                m.home_team_goal AS Goles_Local, 
                m.away_team_goal AS Goles_Visitante, 
                t_away.team_long_name AS Visitante
            FROM Match AS m
            JOIN Team AS t_home ON m.home_team_api_id = t_home.team_api_id
            JOIN Team AS t_away ON m.away_team_api_id = t_away.team_api_id
            WHERE m.home_team_api_id IN (
                SELECT team_api_id FROM Team
                WHERE team_long_name LIKE '%Real Madrid%' OR team_long_name LIKE '%Barcelona%'
            ) AND m.away_team_api_id IN (
                SELECT team_api_id FROM Team
                WHERE team_long_name LIKE '%Real Madrid%' OR team_long_name LIKE '%Barcelona%'
            )''')

,Local,Goles_Local,Goles_Visitante,Visitante
0,FC Barcelona,2,0,Real Madrid CF
1,Real Madrid CF,2,6,FC Barcelona
2,FC Barcelona,1,0,Real Madrid CF
3,Real Madrid CF,0,2,FC Barcelona
4,FC Barcelona,5,0,Real Madrid CF
5,Real Madrid CF,1,1,FC Barcelona
6,Real Madrid CF,1,3,FC Barcelona
7,FC Barcelona,1,2,Real Madrid CF
8,Real Madrid CF,2,1,FC Barcelona
9,FC Barcelona,2,2,Real Madrid CF


##### Promedio y número de Goles en el Clásico de cada Equipo

In [16]:
consulta('''SELECT 
                SUM(CASE WHEN t_home.team_long_name = 'FC Barcelona' THEN m.home_team_goal ELSE m.away_team_goal END) AS suma_goles_barcelona,
                SUM(CASE WHEN t_home.team_long_name = 'Real Madrid' THEN m.home_team_goal ELSE m.away_team_goal END) AS suma_goles_madrid,
                AVG(CASE WHEN t_home.team_long_name = 'FC Barcelona' THEN m.home_team_goal ELSE m.away_team_goal END) AS promedio_goles_barcelona,
                AVG(CASE WHEN t_home.team_long_name = 'Real Madrid' THEN m.home_team_goal ELSE m.away_team_goal END) AS promedio_goles_madrid
            FROM Match AS m
            JOIN Team AS t_home ON m.home_team_api_id = t_home.team_api_id
            JOIN Team AS t_away ON m.away_team_api_id = t_away.team_api_id
            WHERE m.home_team_api_id IN (
                                    SELECT team_api_id FROM Team
                                    WHERE team_long_name LIKE '%Real Madrid%' OR team_long_name LIKE '%Barcelona%'
                                ) AND m.away_team_api_id IN (
                                    SELECT team_api_id FROM Team
                                    WHERE team_long_name LIKE '%Real Madrid%' OR team_long_name LIKE '%Barcelona%'
                                )''')

,suma_goles_barcelona,suma_goles_madrid,promedio_goles_barcelona,promedio_goles_madrid
0,38,30,2.375,1.875


##### Clasificación de la Liga Española en la Temporada 2011/2012

In [17]:
consulta('''SELECT t.team_long_name AS Equipo, SUM(puntos) AS Puntos 
            FROM (
                SELECT 
                    home_team_api_id AS equipo,
                    SUM(
                        CASE 
                            WHEN home_team_goal > away_team_goal THEN 3
                            WHEN home_team_goal = away_team_goal THEN 1
                            ELSE 0 
                        END
                    ) AS puntos
                FROM Match
                WHERE season == '2011/2012' AND league_id == 21518
                GROUP BY home_team_api_id

                UNION ALL

                SELECT 
                    away_team_api_id AS equipo,
                    SUM(
                        CASE 
                            WHEN away_team_goal > home_team_goal THEN 3 
                            WHEN away_team_goal = home_team_goal THEN 1 
                            ELSE 0
                        END
            ) AS puntos
            FROM Match
            WHERE season == '2011/2012' AND league_id == 21518
            GROUP BY away_team_api_id
        ) AS m
        JOIN Team AS t ON m.equipo = t.team_api_id
        GROUP BY Equipo
        ORDER BY Puntos DESC''')

,Equipo,Puntos
0,Real Madrid CF,100
1,FC Barcelona,91
2,Valencia CF,61
3,Málaga CF,58
4,Atlético Madrid,56
5,Levante UD,55
6,CA Osasuna,54
7,RCD Mallorca,52
8,Sevilla FC,50
9,Athletic Club de Bilbao,49


##### Clasificación de todas las Temporadas de cada Liga

In [18]:
consulta('''WITH partidos_completos AS (
                SELECT 
                    league_id,
                    season,
                    home_team_api_id AS equipo_local,
                    away_team_api_id AS equipo_visitante,
                    home_team_goal AS goles_local,
                    away_team_goal AS goles_visitante
                FROM Match
            ),
            puntos_equipos_locales AS (
                SELECT 
                    league_id,
                    season,
                    equipo_local AS equipo,
                    CASE 
                        WHEN goles_local > goles_visitante THEN 3
                        WHEN goles_local = goles_visitante THEN 1
                        ELSE 0
                    END AS puntos
                FROM partidos_completos
            ),
            puntos_equipos_visitantes AS (
                SELECT 
                    league_id,
                    season,
                    equipo_visitante AS equipo,
                    CASE 
                        WHEN goles_visitante > goles_local THEN 3
                        WHEN goles_visitante = goles_local THEN 1
                        ELSE 0
                    END AS puntos
                FROM partidos_completos
            )
            SELECT 
                l.name AS Liga,
                m.season AS Temporada,
                t.team_long_name AS Equipo,
                SUM(puntos) AS Puntos
            FROM (
                SELECT * FROM puntos_equipos_locales
                UNION ALL
                SELECT * FROM puntos_equipos_visitantes
            ) AS m
            JOIN League AS l ON m.league_id = l.id
            JOIN Team AS t ON m.equipo = t.team_api_id
            GROUP BY league_id, season, equipo
            ORDER BY league_id, season, Puntos DESC;
            ''')

,Liga,Temporada,Equipo,Puntos
0,Belgium Jupiler League,2008/2009,Standard de Liège,77
1,Belgium Jupiler League,2008/2009,RSC Anderlecht,77
2,Belgium Jupiler League,2008/2009,KAA Gent,59
3,Belgium Jupiler League,2008/2009,Club Brugge KV,59
4,Belgium Jupiler League,2008/2009,SV Zulte-Waregem,55
...,...,...,...,...
1476,Switzerland Super League,2015/2016,FC Thun,41
1477,Switzerland Super League,2015/2016,FC St. Gallen,38
1478,Switzerland Super League,2015/2016,FC Vaduz,36
1479,Switzerland Super League,2015/2016,Lugano,35


##### Líderes de cada Temporada de las 5 Grandes Ligas (España, Inglaterra, Italia, Alemania y Francia)

In [19]:
consulta('''WITH partidos_completos AS (
                SELECT 
                    league_id,
                    season,
                    home_team_api_id AS equipo_local,
                    away_team_api_id AS equipo_visitante,
                    home_team_goal AS goles_local,
                    away_team_goal AS goles_visitante
                FROM Match AS m
                JOIN League AS l ON m.league_id = l.id
                JOIN Country AS c ON l.country_id = c.id
                WHERE c.name IN ('Spain', 'England', 'Italy', 'Germany', 'France')
            ),
            puntos_equipos_locales AS (
                SELECT 
                    league_id,
                    season,
                    equipo_local AS equipo,
                    CASE 
                        WHEN goles_local > goles_visitante THEN 3
                        WHEN goles_local = goles_visitante THEN 1
                        ELSE 0
                    END AS puntos
                FROM partidos_completos
            ),
            puntos_equipos_visitantes AS (
                SELECT 
                    league_id,
                    season,
                    equipo_visitante AS equipo,
                    CASE 
                        WHEN goles_visitante > goles_local THEN 3
                        WHEN goles_visitante = goles_local THEN 1
                        ELSE 0
                    END AS puntos
                FROM partidos_completos
            ),
            puntos_totales AS (
                SELECT * FROM puntos_equipos_locales
                UNION ALL
                SELECT * FROM puntos_equipos_visitantes
            )
            SELECT 
                l.name AS Liga,
                m.season AS Temporada,
                t.team_long_name AS Equipo,
                Puntos 
            FROM (
                SELECT 
                    league_id,
                    season,
                    equipo,
                    Puntos,
                    ROW_NUMBER() OVER (PARTITION BY league_id, season ORDER BY Puntos DESC) AS rn
                FROM (
                    SELECT 
                        league_id,
                        season,
                        equipo,
                        SUM(puntos) AS Puntos
                    FROM puntos_totales
                    GROUP BY league_id, season, equipo
                ) AS puntos_por_equipo
            ) AS m
            JOIN League AS l ON m.league_id = l.id
            JOIN Team AS t ON m.equipo = t.team_api_id
            WHERE rn = 1
            ORDER BY league_id, season;
            ''')

,Liga,Temporada,Equipo,Puntos
0,England Premier League,2008/2009,Manchester United,90
1,England Premier League,2009/2010,Chelsea,86
2,England Premier League,2010/2011,Manchester United,80
3,England Premier League,2011/2012,Manchester City,89
4,England Premier League,2012/2013,Manchester United,89
5,England Premier League,2013/2014,Manchester City,86
6,England Premier League,2014/2015,Chelsea,87
7,England Premier League,2015/2016,Leicester City,81
8,France Ligue 1,2008/2009,Girondins de Bordeaux,80
9,France Ligue 1,2009/2010,Olympique de Marseille,78


##### Comparación del número de victorias como local y como visitante.

In [20]:
consulta('''SELECT 
                SUM(CASE WHEN home_team_goal > away_team_goal THEN 1 ELSE 0 END) AS victorias_local,
                SUM(CASE WHEN home_team_goal = away_team_goal THEN 1 ELSE 0 END) AS empates,
                SUM(CASE WHEN home_team_goal < away_team_goal THEN 1 ELSE 0 END) AS victorias_visitante,
                ROUND((SUM(CASE WHEN home_team_goal > away_team_goal THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2) AS porcentaje_local,
                ROUND((SUM(CASE WHEN home_team_goal = away_team_goal THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2) AS porcentaje_empate,
                ROUND((SUM(CASE WHEN home_team_goal < away_team_goal THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2) AS porcentaje_visitante
            FROM Match;''')

,victorias_local,empates,victorias_visitante,porcentaje_local,porcentaje_empate,porcentaje_visitante
0,11917,6596,7466,45.87,25.39,28.74
